# Data Distribution with PartitionedArray

This notebook demonstrates how HPXPy distributes data across multiple localities using `PartitionedArray` (backed by `hpx::partitioned_vector`).

Key concepts:
- **Partitions**: Contiguous chunks of data, each residing on one locality
- **Distribution**: How partitions are assigned to localities
- **Distributed operations**: Element-wise and reduction operations that work across partitions

When running with N localities, each locality physically owns a portion of the data. Operations execute locally on each partition and combine results as needed.

In [ ]:
%%writefile _distribution_demo_worker.py
"""Data distribution demo: how PartitionedArray partitions data across localities."""
import sys
import numpy as np
import hpxpy as hpx
from hpxpy.launcher import init_from_args

init_from_args()

my_id = hpx.locality_id()
num_locs = hpx.num_localities()
print(f"[Locality {my_id}/{num_locs}] Started with {hpx.num_threads()} threads")

# ============================================================
# 1. Creating Distributed Arrays
# ============================================================
print(f"\n{'='*50}")
print("1. PartitionedArray Creation")
print(f"{'='*50}")

# Create arrays using different factory functions
arr_arange = hpx.partitioned_arange(0, 20)
arr_zeros = hpx.partitioned_zeros(10)
arr_ones = hpx.partitioned_ones(10)
arr_full = hpx.partitioned_full(10, 3.14)
arr_numpy = hpx.partitioned_from_numpy(np.array([10.0, 20.0, 30.0, 40.0, 50.0]))

if my_id == 0:
    print(f"  partitioned_arange(0, 20): {arr_arange.num_partitions} partitions, distributed={arr_arange.is_distributed}")
    print(f"  partitioned_zeros(10):     {arr_zeros.num_partitions} partitions, distributed={arr_zeros.is_distributed}")
    print(f"  partitioned_ones(10):      {arr_ones.num_partitions} partitions, distributed={arr_ones.is_distributed}")
    print(f"  partitioned_full(10, pi):  {arr_full.num_partitions} partitions, distributed={arr_full.is_distributed}")
    print(f"  partitioned_from_numpy:    {arr_numpy.num_partitions} partitions, distributed={arr_numpy.is_distributed}")

hpx.barrier("after_creation")

# ============================================================
# 2. Distributed Reductions
# ============================================================
print(f"\n{'='*50}")
print("2. Distributed Reductions")
print(f"{'='*50}")

# Create a larger distributed array for meaningful reductions
data = hpx.partitioned_from_numpy(np.arange(1000, dtype=np.float64))

if my_id == 0:
    print(f"  Array: partitioned_from_numpy(arange(1000))")
    print(f"  Partitions: {data.num_partitions}")
    print(f"  Distributed: {data.is_distributed}")
    print()

# All reductions work across all localities
s = hpx.distributed_sum(data)
m = hpx.distributed_mean(data)
mn = hpx.distributed_min(data)
mx = hpx.distributed_max(data)
v = hpx.distributed_var(data)
sd = hpx.distributed_std(data)

if my_id == 0:
    print(f"  sum  = {s}")
    print(f"  mean = {m}")
    print(f"  min  = {mn}")
    print(f"  max  = {mx}")
    print(f"  var  = {v:.4f}")
    print(f"  std  = {sd:.4f}")

    # Verify against NumPy
    ref = np.arange(1000, dtype=np.float64)
    assert s == np.sum(ref), f"sum mismatch: {s} vs {np.sum(ref)}"
    assert m == np.mean(ref), f"mean mismatch: {m} vs {np.mean(ref)}"
    assert mn == np.min(ref), f"min mismatch: {mn} vs {np.min(ref)}"
    assert mx == np.max(ref), f"max mismatch: {mx} vs {np.max(ref)}"
    print("  (All verified against NumPy)")

hpx.barrier("after_reductions")

# ============================================================
# 3. Converting Back to NumPy
# ============================================================
print(f"\n{'='*50}")
print("3. Conversion to NumPy")
print(f"{'='*50}")

small = hpx.partitioned_from_numpy(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
local_copy = small.to_numpy()
if my_id == 0:
    print(f"  PartitionedArray → NumPy: {local_copy}")
    print(f"  Type: {type(local_copy)}")

hpx.barrier("done")
if my_id == 0:
    print("\nAll localities completed successfully.")
hpx.finalize()

## Launch Distributed Execution

The worker script runs on multiple localities. PartitionedArrays are automatically split across the localities.

In [ ]:
from hpxpy.launcher import launch_localities

launch_localities(
    "_distribution_demo_worker.py",
    num_localities=2,
    threads_per_locality=2,
    verbose=True,
)

## How Partitioning Works

When `partitioned_from_numpy(data)` is called with N localities:

```
NumPy array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

                    ┌─── partitioned_from_numpy ───┐
                    ▼                               ▼
         Locality 0                      Locality 1
    ┌──────────────────┐           ┌──────────────────┐
    │ Partition 0      │           │ Partition 1      │
    │ [0, 1, 2, 3, 4] │           │ [5, 6, 7, 8, 9]  │
    └──────────────────┘           └──────────────────┘
```

### Distributed Reduction Flow

```
    Locality 0              Locality 1
    sum([0..4]) = 10        sum([5..9]) = 35
         │                       │
         └───── combine ─────────┘
                    │
              global sum = 45
```

This happens automatically when you call `distributed_sum()`, `distributed_mean()`, etc.

### API Summary

| Creation | Description |
|----------|-------------|
| `partitioned_zeros(n)` | Distributed zeros |
| `partitioned_ones(n)` | Distributed ones |
| `partitioned_full(n, val)` | Distributed fill |
| `partitioned_arange(start, stop)` | Distributed range |
| `partitioned_from_numpy(arr)` | Distribute existing data |

| Property | Description |
|----------|-------------|
| `.num_partitions` | Number of data partitions |
| `.is_distributed` | True if multi-locality |
| `.to_numpy()` | Gather all data to local NumPy array |

In [ ]:
import os
os.remove("_distribution_demo_worker.py")
print("Cleaned up worker script.")